Illiquid Asset Volume Trends
### By performing these quantitative analyses on asset volume data, traders and analysts can gain insights into market liquidity and trading patterns, helping to identify more opportune times for executing large trades in illiquid assets, thereby potentially minimizing market impact and cost basis.

#### Data Loading 

In [6]:
#load libraries and dependencies 

import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from pathlib import Path
import datetime as dt
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import json
import os



In [7]:
#env variables
load_dotenv('api_keys.env')
api_keys = os.getenv('ALPACA_API_KEY')
secret_key = os.getenv('ALPACA_SECRET_KEY')

In [8]:
#initiate variables api object
ticker = 'TSLA'
today = dt.datetime.now().isoformat()
timeframe = '1D'
start_date = pd.Timestamp('2019-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2024-01-01', tz='America/New_York').isoformat()
api = tradeapi.REST(
    api_keys,
    secret_key,
    api_version='v2'
)

In [9]:
#alpaca api
df = api.get_bars(
    ticker,
    timeframe,
    start=start_date,
    end=end_date
).df

# #set index and drop na values
df.dropna(inplace=True)
df.index = df.index.date
df.index.name = 'date'
df.reset_index(inplace=True)
df.rename_axis(index={'Letter': 'NewLetter'}, inplace=True)
df['date'] = pd.to_datetime(df['date'])

# #set daily, monthly and yearly columns
df['day'] = df['date'].dt.day_name()
df['week'] = df['date'].dt.isocalendar().week
df['month'] = df['date'].dt.month_name()
df['year'] = df['date'].dt.year

df.head()

,date,close,high,low,trade_count,open,volume,vwap,day,week,month,year
0,2019-01-02,310.12,315.13,298.8000,144873,306.10,11836650,307.640086,Wednesday,1,January,2019
1,2019-01-03,300.36,309.40,297.3800,81545,307.00,7087451,302.152581,Thursday,1,January,2019
2,2019-01-04,317.69,318.00,302.7300,84464,306.00,7538704,311.967824,Friday,1,January,2019
3,2019-01-07,334.96,336.74,317.7508,87650,321.72,7660432,330.752600,Monday,2,January,2019
4,2019-01-08,335.35,344.01,327.0200,75881,341.96,7212138,335.523364,Tuesday,2,January,2019


### Data Preprocessing

In [10]:
# #seperate data by price and volume and group by day, month and year 
df_data = df.drop(columns=['open', 'high', 'low',])
df_data['date'] = pd.to_datetime(df_data['date'])
df_data.set_index('date', inplace=True)
df_data = df_data.reindex(columns=['week', 'year', 'month', 'day', 'volume', 'close', 'vwap', 'trade_count'])
df_data.head()


,week,year,month,day,volume,close,vwap,trade_count
date,,,,,,,,
2019-01-02,1,2019,January,Wednesday,11836650,310.12,307.640086,144873
2019-01-03,1,2019,January,Thursday,7087451,300.36,302.152581,81545
2019-01-04,1,2019,January,Friday,7538704,317.69,311.967824,84464
2019-01-07,2,2019,January,Monday,7660432,334.96,330.752600,87650
2019-01-08,2,2019,January,Tuesday,7212138,335.35,335.523364,75881


In [11]:
# separate data 
daily_close = df['close'].groupby(df['date'])
daily_volume = df['volume'].groupby(df['date'])
daily_vwap = df['vwap'].groupby(df['date'])

#volume highs and lows 
weekly_vol = df_data['volume'].resample('W').agg(['max', 'min'])
monthly_vol = df_data['volume'].resample('M').agg(['max', 'min'])
yearly_vol = df_data['volume'].resample('A').agg(['max', 'min'])

/var/folders/jf/lcpt8z3d2gb1_qpgt0d_xdw00000gn/T/ipykernel_47606/3318367505.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_vol = df_data['volume'].resample('M').agg(['max', 'min'])
/var/folders/jf/lcpt8z3d2gb1_qpgt0d_xdw00000gn/T/ipykernel_47606/3318367505.py:9: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_vol = df_data['volume'].resample('A').agg(['max', 'min'])


In [18]:
#weekly high / low 
weekly_days = df_data[df_data['volume'].isin(weekly_vol['max'].tolist() + weekly_vol['min'].tolist())]

def label_volume(row):
    if row['volume'] in weekly_vol['max'].tolist():
        return 'high'
    elif row['volume'] in weekly_vol['min'].tolist():
        return 'low'
    else:
        return 'unknown'
        
weekly_days['Label'] = weekly_days.apply(label_volume, axis=1)

/var/folders/jf/lcpt8z3d2gb1_qpgt0d_xdw00000gn/T/ipykernel_47606/1900042430.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekly_days['Label'] = weekly_days.apply(label_volume, axis=1)


In [16]:
#monthy high / low
monthly_days = df_data[df_data['volume'].isin(monthly_vol['max'].tolist() + monthly_vol['min'].tolist())]

def label_volume(row):
    if row['volume'] in monthly_vol['max'].tolist():
        return 'high'
    elif row['volume'] in monthly_vol['min'].tolist():
        return 'low'
    else:
        return 'unknown'
        
monthly_days['Label'] = monthly_days.apply(label_volume, axis=1)

monthly_days

/var/folders/jf/lcpt8z3d2gb1_qpgt0d_xdw00000gn/T/ipykernel_47606/3990095358.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_days['Label'] = monthly_days.apply(label_volume, axis=1)


,week,year,month,day,volume,close,vwap,trade_count,Label
date,,,,,,,,,
2019-01-17,3,2019,January,Thursday,3766877,347.31,347.740889,41335,low
2019-01-18,3,2019,January,Friday,24359859,302.26,311.621360,284608,high
2019-02-15,7,2019,February,Friday,4009317,307.88,306.617897,42988,low
2019-02-27,9,2019,February,Wednesday,11307356,314.74,310.679514,117960,high
2019-03-01,9,2019,March,Friday,23064224,294.79,296.910074,262716,high
...,...,...,...,...,...,...,...,...,...
2023-10-19,42,2023,October,Thursday,170944596,220.11,221.976513,2329342,high
2023-11-15,46,2023,November,Wednesday,150391782,242.84,242.711759,1553354,high
2023-11-24,47,2023,November,Friday,65125204,235.45,236.399808,738518,low


In [17]:
#yearly high / low
yearly_days = df_data[df_data['volume'].isin(yearly_vol['max'].tolist() + yearly_vol['min'].tolist())]

def label_volume(row):
    if row['volume'] in yearly_vol['max'].tolist():
        return 'high'
    elif row['volume'] in yearly_vol['min'].tolist():
        return 'low'
    else:
        return 'unknown'

yearly_days['Label'] = yearly_days.apply(label_volume, axis=1)

yearly_days

/var/folders/jf/lcpt8z3d2gb1_qpgt0d_xdw00000gn/T/ipykernel_47606/1195643890.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yearly_days['Label'] = yearly_days.apply(label_volume, axis=1)


,week,year,month,day,volume,close,vwap,trade_count,Label
date,,,,,,,,,
2019-10-24,43,2019,October,Thursday,30428851,299.68,297.598962,345503,high
2019-11-29,48,2019,November,Friday,2565414,329.94,329.513208,36298,low
2020-08-05,32,2020,August,Wednesday,5054464,1485.02,1485.014821,178471,low
2020-12-18,51,2020,December,Friday,300234026,658.34,683.754422,2078694,high
2021-03-05,9,2021,March,Friday,90222134,597.95,583.972394,2552198,high
2021-08-11,32,2021,August,Wednesday,10174556,707.82,708.540322,256403,low
2022-08-18,33,2022,August,Thursday,16135100,908.61,912.667690,490142,low
2022-12-29,52,2022,December,Thursday,226370514,121.82,120.567766,1902592,high
2023-01-27,4,2023,January,Friday,309508609,177.90,172.967094,2617785,high


In [33]:
#plot weekly high and low volume
weekly_days.hvplot.line(
    x='date',
    y='volume',
    xlabel='Date',
    ylabel='Volume',
    title='Weekly High and Low Volume'
).opts(yformatter='%.0f')

:Curve   [date]   (volume)

In [32]:
#plot monthly high and low volume
monthly_days.hvplot.line(
    x='date',
    y='volume',
    xlabel='Date',
    ylabel='Volume',
    title='Monthly High and Low Volume'
).opts(yformatter='%.0f')

:Curve   [date]   (volume)

In [34]:
#plot yearly high and low volume
yearly_days.hvplot.line(
    x='date',
    y='volume',
    xlabel='Date',
    ylabel='Volume',
    title='Yearly High and Low Volume'
).opts(yformatter='%.0f')

:Curve   [date]   (volume)

### Correlation 

In [44]:
### correlation coefficiants between mtrics
weekly_correlation = weekly_days[['volume', 'close', 'vwap', 'trade_count']].corr()
monthly_correlation = monthly_days[['volume', 'close', 'vwap', 'trade_count']].corr()
yearly_correlation = yearly_days[['volume', 'close', 'vwap', 'trade_count']].corr()
print(f'weekly correlation')
display(weekly_correlation)
print(f'monthly correlation')
display(monthly_correlation)
print(f'yearly correlation')
display(yearly_correlation)

weekly correlation


,volume,close,vwap,trade_count
volume,1.000000,-0.441984,-0.441011,0.854108
close,-0.441984,1.000000,0.999341,-0.133155
vwap,-0.441011,0.999341,1.000000,-0.131777
trade_count,0.854108,-0.133155,-0.131777,1.000000


monthly correlation


,volume,close,vwap,trade_count
volume,1.000000,-0.394104,-0.388492,0.817017
close,-0.394104,1.000000,0.998489,-0.094490
vwap,-0.388492,0.998489,1.000000,-0.091782
trade_count,0.817017,-0.094490,-0.091782,1.000000


yearly correlation


,volume,close,vwap,trade_count
volume,1.000000,-0.414565,-0.404856,0.842289
close,-0.414565,1.000000,0.999730,-0.359526
vwap,-0.404856,0.999730,1.000000,-0.359384
trade_count,0.842289,-0.359526,-0.359384,1.000000


In [52]:
weekly_correlation.hvplot.heatmap(
    title='Weekly Correlation Heatmap',
    cmap='viridis'
)

:HeatMap   [columns,index]   (value)

In [56]:
monthly_correlation.hvplot.heatmap(
    title='Monthly Correlation Heatmap',
    C = monthly_correlation,
    cmap='viridis'
)

:HeatMap   [columns,index]   (value)

In [61]:
yearly_correlation.hvplot.heatmap(
    title='Yearly Correlation Heatmap',
    C = yearly_correlation,
    cmap='viridis'
)

:HeatMap   [columns,index]   (value)

### Analysis

### Visualization